## Import

In [2]:
import glob
from math import sqrt
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
import geopandas as gpd
# import geoparquet as gpq

from math import *
from shapely.geometry import Point

import dask.dataframe as dd
from dask.diagnostics import ProgressBar
# import dask_geopandas as dg



from pathlib import Path
import os
import glob

import re
from tqdm import tqdm

%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Data

In [3]:
%%time
# sa
sa = gpd.read_file('C:/Users/alexd/Dropbox/Work/Repository/FFE/data/shapefile/SA2_input.shp')
sa = sa[['BldgFloor_', 'PopDensity', 'NumberBldg', 'geometry', 'SA2_ID']] # drop column not used in caluclation of Ignition
sa = gpd.GeoDataFrame(sa, crs="EPSG:2193")

# pga 
pga = pd.read_parquet('C:/Users/alexd/Dropbox/Work/Repository/OpenQuake/output/PGAxy.parquet')
pgaGeo = gpd.GeoDataFrame(pga, geometry=gpd.points_from_xy(pga.lon, pga.lat))#, crs="EPSG:2193"))

del pga

Wall time: 3min 6s


In [8]:
%%time
data = gpd.sjoin(pgaGeo, sa, op="within")

Wall time: 25min 13s


## Ignition calculations and save join shapefile

In [35]:
from numba import jit

gmv_PGA = data.gmv_PGA.values
PopDensity = data.PopDensity
BldFloor = data.BldgFloor_.values
NbrBld = data.NumberBldg.values

@jit
def ignition_district(PGAs, PD, SF):
    Pign = np.exp(-6.755 + (8.463 * PGAs) + (98.4e-6 * PD) + (152.3e-6 * SF)) / (1 + np.exp(-6.755 + (8.463 * PGAs) + (98.4e-6 * PD) + (152.3e-6 * SF)))
    return Pign


@jit
def ignition_buildings(IgnDistrict, NBbld):
    PignBld = 1 - np.power(1-IgnDistrict, 1/NBbld)
    return PignBld

In [36]:
%%time
Pignition_district = ignition_district(gmv_PGA, PopDensity, BldFloor)
Pignition_buildings = ignition_buildings(Pignition_district, NbrBld)

Wall time: 3.37 s


In [47]:
data['IgnProb'] = Pignition_buildings
joinSA_PGA = data[['event_id', 'site_id', 'SA2_ID','gmv_PGA', 'lon', 'lat', 'geometry','IgnProb']]

In [52]:
%%time
joinSA_PGA.to_file('C:/Users/alexd/Dropbox/Work/InOutRepoData/FFE/ProbaScenariosInput/joinSA_PGA.shp')

KeyboardInterrupt: 